In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
import zipfile
with zipfile.ZipFile('/content/glove.6B.zip','r') as zip:
  zip.extractall('/content/')

!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
!gunzip -k /content/download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz
!mkdir lines
!split -a 3 -l 100000 download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en lines/lines-

--2020-06-27 06:39:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-27 06:39:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-27 06:39:54--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [2]:
import os
files = os.listdir('/content/lines')
print(len(files))

4415


In [3]:
files = files[:100]
min_len = 40
max_len = 100
context = []
response = []

for f in files:
  file = open('/content/lines/'+f)
  i = 0
  for j,line in enumerate(file):
      if len(line)<min_len or len(line)>max_len:
        continue
      if i%2==0:
          context.append(line)
          i+=1
      else:
          response.append("awwal "+line+" aakhir")
          i+=1


print(len(response),len(context))
context = context[:533232]

1083217 1083267


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
MAX_SEQ_LENGTH = 20
EMBEDDING_DIM = 50
vocab_size = 20000
tokenizer = Tokenizer(num_words=vocab_size,oov_token="<oov>")
tokenizer.fit_on_texts(context+response)

word_index = {word:index for word,index in tokenizer.word_index.items() if index<vocab_size }
index_to_word = { index:word for word,index in word_index.items()}

context_seq = tokenizer.texts_to_sequences(context)
response_seq = tokenizer.texts_to_sequences(response)

In [6]:

encoder_inputs = []
decoder_inputs = []
decoder_outputs = []
for i in range(len(context_seq)):
    if not(len(context_seq[i])>MAX_SEQ_LENGTH or len(response_seq[i])-1>MAX_SEQ_LENGTH):
        encoder_inputs.append(context_seq[i])
        decoder_inputs.append(response_seq[i][:-1])
        decoder_outputs.append(response_seq[i][1:])

encoder_inputs = pad_sequences(encoder_inputs,maxlen = MAX_SEQ_LENGTH,padding='post',truncating='post')
decoder_inputs = pad_sequences(decoder_inputs,maxlen = MAX_SEQ_LENGTH,padding='post',truncating='post')
decoder_outputs = pad_sequences(decoder_outputs,maxlen = MAX_SEQ_LENGTH,padding='post',truncating='post')

print(len(encoder_inputs),len(decoder_inputs),len(decoder_outputs))

527500 527500 527500


In [7]:
print(decoder_outputs[:5])

[[   51    30  1146    12   643  9921    23    77     3     0     0     0
      0     0     0     0     0     0     0     0]
 [  317    45   208    24   217    15    19   929     3     0     0     0
      0     0     0     0     0     0     0     0]
 [   39    31   565     6 18887    55    28  4049   138   146  2243     3
      0     0     0     0     0     0     0     0]
 [   46     7    41   128    10   150     1     1  4085    23     4 16068
     22   538     3     0     0     0     0     0]
 [   31    38     5  3161   113    49    16   531     3     0     0     0
      0     0     0     0     0     0     0     0]]


In [8]:
import os
import numpy as np
embeddings_index = {}
f = open(os.path.join('/content', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.randn(1,EMBEDDING_DIM)

from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            #mask_zero = True,
                            trainable=False)

Found 400000 word vectors.


In [9]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model

In [10]:
# Defined shared layers as global variables
repeator = RepeatVector(MAX_SEQ_LENGTH)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "softmax")
dotor = Dot(axes = 1)

In [11]:
def one_step_attention(a, s_prev):
    #calculating context vector
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])
    e = densor1(concat)
    alphas = densor2(e)
    context = dotor([alphas,a])
    
    return context

In [12]:
def encoder_decoder_model():
    hidden_dim = 512
    
    #encoder
    encoder_inputs = Input(shape=(MAX_SEQ_LENGTH,))
    emb_en_input = embedding_layer(encoder_inputs)
    #encoder_lstm = LSTM(hidden_dim,return_sequences=True,return_state=True)
    encoder_gru = GRU(hidden_dim,return_sequences=True,return_state=True)
    #whole_encoder_outputs,state_h,state_c = encoder_lstm(emb_en_input)
    whole_encoder_outputs,state_h = encoder_gru(emb_en_input)
    #encoder_states = [state_h,state_c]
    encoder_states = [state_h]

    
    #getting context vector
    a = whole_encoder_outputs #for one step attention
    s = state_h
    #c = state_c
    #decoder model for training
    decoder_inputs = Input(shape=(MAX_SEQ_LENGTH,))
    emb_de_input = embedding_layer(decoder_inputs)
    #decoder_lstm = LSTM(hidden_dim,return_state=True)
    decoder_gru = GRU(hidden_dim,return_state=True)
    decoder_dense  = Dense(vocab_size,activation="softmax")
    outputs = []
    for t in range(MAX_SEQ_LENGTH): #number of output time steps
      context = one_step_attention(a,s) 
      concatenated_input = concatenator([context,emb_de_input[:,t:t+1]])
      decoder_output,s = decoder_gru(concatenated_input,initial_state=[s])
      decoder_output = decoder_dense(decoder_output)
      outputs.append(decoder_output)
    decoder_outputs = outputs#complete output
    

    #model for training
    model = Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)
   
    return model

In [13]:
model = encoder_decoder_model()

In [14]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 50)       1000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       [(None, 20, 512), (N 866304      embedding[0][0]                  
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 20, 512)      0           gru[0][1]                    

In [16]:
print(encoder_inputs.shape)

(527500, 20)


In [17]:
encoder_inputs = encoder_inputs[:100000]
decoder_inputs = decoder_inputs[:100000]
decoder_outputs = decoder_outputs[:100000]

In [18]:
def data_generator(batch_size = 64):
    i = 0
    indexes = np.arange(len(encoder_inputs)//batch_size)
    while True:
        if(i>=len(indexes)):
            i = 0
            np.random.shuffle(indexes)
        X1  = encoder_inputs[indexes[i]*batch_size:(indexes[i]+1)*batch_size]
        X2  = decoder_inputs[indexes[i]*batch_size:(indexes[i]+1)*batch_size]
        Y  =  decoder_outputs[indexes[i]*batch_size:(indexes[i]+1)*batch_size]
        Y = list(Y.swapaxes(0,1))
        i+=1
        yield([X1,X2],list(Y))

In [19]:
batch = 64
datagen = data_generator(batch_size=batch)

In [20]:
model.fit(datagen,steps_per_epoch=len(decoder_inputs)//batch,epochs=10)

Epoch 1/10
1562/1562 [==============================] - 442s 283ms/step - loss: 74.1346 - dense_2_loss: 5.4838 - dense_2_1_loss: 6.2422 - dense_2_2_loss: 6.3892 - dense_2_3_loss: 6.5304 - dense_2_4_loss: 6.5747 - dense_2_5_loss: 6.6452 - dense_2_6_loss: 6.6239 - dense_2_7_loss: 6.3520 - dense_2_8_loss: 5.5996 - dense_2_9_loss: 4.5714 - dense_2_10_loss: 3.5233 - dense_2_11_loss: 2.7082 - dense_2_12_loss: 2.0443 - dense_2_13_loss: 1.5191 - dense_2_14_loss: 1.1137 - dense_2_15_loss: 0.8217 - dense_2_16_loss: 0.5777 - dense_2_17_loss: 0.3854 - dense_2_18_loss: 0.2591 - dense_2_19_loss: 0.1697 - dense_2_accuracy: 0.0977 - dense_2_1_accuracy: 0.0520 - dense_2_2_accuracy: 0.0502 - dense_2_3_accuracy: 0.0483 - dense_2_4_accuracy: 0.0470 - dense_2_5_accuracy: 0.0465 - dense_2_6_accuracy: 0.0501 - dense_2_7_accuracy: 0.0953 - dense_2_8_accuracy: 0.2121 - dense_2_9_accuracy: 0.3725 - dense_2_10_accuracy: 0.5196 - dense_2_11_accuracy: 0.6381 - dense_2_12_accuracy: 0.7313 - dense_2_13_accuracy: 0.8

In [21]:
model.save('/content/encoder_decoder_model_gru.h5')
from google.colab import files
files.download('/content/encoder_decoder_model_gru.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/encoder_decoder_model_moretrained.h5')

In [22]:
def decode_sequence_argmax(input_seq):

  target_seq = np.zeros((1,MAX_SEQ_LENGTH+1))
  target_seq[0,0] = word_index["awwal"]
  target_seq = pad_sequences(target_seq,maxlen=MAX_SEQ_LENGTH,padding='post',truncating='post')
  fed_seq = [input_seq,target_seq]
  stop_condition = False
  decoded_sentence = ''
  for i in range(MAX_SEQ_LENGTH):
    output = model.predict(fed_seq)
    index_of_word = np.argmax(output[i])
    if index_of_word==0:
      predicted_word = 'zero'
    else:
      predicted_word = index_to_word[index_of_word]

    if predicted_word == "aakhir":
      break
    else:
      decoded_sentence+=" "+predicted_word

    target_seq[0,i+1] = index_of_word
    fed_seq = [input_seq,target_seq]

  return decoded_sentence

In [62]:
def decode_sequence_beam_search(input_seq,beam_width=3):
  index_to_word[0] = ""
  target_seq = np.zeros((beam_width,MAX_SEQ_LENGTH))#to store target sequences at different time steps
  is_complete = []#if the given sequence has been ended by "aakhir" token
  fed_seq = []#sequence to be fed to predict output
  scores = np.zeros((beam_width,))#beam scores
  for i in range(beam_width):
    target_seq[i,0] = word_index["awwal"]
    fed_seq.append([input_seq,target_seq[i:i+1]])
    is_complete.append(False)

  for i in range(MAX_SEQ_LENGTH-1):
    all_candidates = []#all possible beam_width*beam_width candidates at a given time step
    all_scores = []#their scores
    all_bool = []#if the given seq is complete
    for j in range(beam_width):
      if is_complete[j]: #if the seq is complete 
        all_candidates.append(np.array(target_seq[j]))
        all_scores+=[scores[j]]
        all_bool+=[is_complete[j]]
        continue
      output = model.predict(fed_seq[j])[i]

      tmp_scores = scores[j]+np.log(output).reshape(vocab_size,) #(scores[j]*(i+1)+np.log(output).reshape(vocab_size,))/(i+2)  #length_normalization
      indexes = np.argsort(-1*tmp_scores)[:beam_width]#top k candidates
      scores_of_top = np.array([tmp_scores[i] for i in indexes])
      tmp_bool = [index_to_word[i]=="aakhir" for i in indexes]

      for index in indexes:
        new_candidate = np.array(target_seq[j])
        new_candidate[i+1] = index

        all_candidates.append(new_candidate)
      
      all_scores+=list(scores_of_top)
      all_bool+=tmp_bool

    if i==0:
      sorted_scores_indexes = np.arange(0,beam_width) #first k candidates bcoz symmetry imposed by awwal
    else:
      sorted_scores_indexes = np.argsort(-1*np.array(all_scores))[:beam_width] #top k candidates from all beams

    target_seq = [all_candidates[i] for i in sorted_scores_indexes]
    scores = [all_scores[i] for i in sorted_scores_indexes]
    is_complete = [all_bool[i] for i in sorted_scores_indexes]

    fed_seq = [[input_seq,np.array(t).reshape(1,MAX_SEQ_LENGTH)] for t in target_seq]

  return target_seq,scores

[False, False, True, False, False, False]


In [60]:
inputs = ["I AM EXTREMELY SAD TODAY"]#["YOU BETTER GET SOME WORK TO DO"]#["can ai overturn bureaucracy"]#,"give short outputs","motherfucker"]
for input in inputs:
  input = input.split()
  input = tokenizer.texts_to_sequences([input])[0]
  input = pad_sequences([input],maxlen=20,padding='post')
  print(decode_sequence_argmax(input))

 i can't see this guy's like this is a good of two years


 i do know you time to the computer and <oov>
 i after know you know you will the tired place betray
 it hold you're mess sand him the man
 get negotiator you like you got a way reward

In [63]:
inputs = ["i hate ai"]#["can ai overturn bureaucracy","give short outputs","motherfucker","is the project complete"]
for input in inputs:
  input = input.split()
  input = tokenizer.texts_to_sequences([input])[0]
  input = pad_sequences([input],maxlen=20,padding='post')
  outputs,scores = decode_sequence_beam_search(input,beam_width=3)
  for i in range(len(scores)):
    s = ' '.join([index_to_word[t] for t in outputs[i]])
    print(s,scores[i])

awwal i don't wanna be a good time to tell aakhir          -16.349768
awwal i don't wanna be a good time to do this aakhir         -17.712975
awwal i don't wanna be a good time but if you did to do aakhir      -20.26361
